# `Russian Troll Tweets Part-03 NLP pre-processing`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

**`Help Support Channel: Patreon @mrfugudatasci`**
    
**`Or Consider: Buy Me A Coffee @mrfugudatasci`**

**PLEASE READ THIS:**

**`In order to use the CSV, you will need to start from the first notebook due to size of file`**

+ These are sequential videos doing steps to get to this point. You can directly download from Kaggle. I showed how to do this in a previous video. [Kaggle api](https://youtu.be/mmxbyfhG6Ss)

In [2]:
import pandas as pd
import numpy as np
import re

# plotting image with word cloud
from PIL import Image                    # for our images
import numpy as np                       # convert image to np arrays
from wordcloud import WordCloud,ImageColorGenerator,STOPWORDS # generate word cloud
import matplotlib.pyplot as plt          # plotting
import seaborn as sns

# nlp
import nltk                              # text processing
from nltk import word_tokenize           # split sentence into list of words
from nltk.corpus import stopwords        # remove: and,it,i,etc
import string                            # remove punctuation
from collections import defaultdict      # dictionary with values as lists
from nltk.stem import WordNetLemmatizer  # remove word endings etc

%matplotlib inline

In [63]:
troll_02=pd.read_csv('troll_tweets_vid02.csv')

troll_02=troll_02.drop(['content','Unnamed: 0'],axis = 1)

troll_02.tail()

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http
2973365,_YOUR_LIFESTYLE,Unknown,Uzbek,9/7/2015 1:31,60,4,412,Other_Tweet,Russian,0,NonEnglish,Санкционный пинг-понг
2973366,_YOUR_LIFESTYLE,Unknown,Russian,9/8/2015 14:49,60,4,416,Other_Tweet,Russian,0,NonEnglish,Горизонтальный кальян NanoSmoke
2973367,_YOUR_LIFESTYLE,Unknown,Russian,9/9/2015 18:52,60,4,418,Other_Tweet,Russian,0,NonEnglish,Фотограф Kai-Hendrik Schroeder
2973368,_YOUR_LIFESTYLE,Unknown,Russian,9/9/2015 22:26,60,4,419,Other_Tweet,Russian,0,NonEnglish,К чему стремится компания Netflix (США)
2973369,_YOUR_LIFESTYLE,Unknown,Russian,9/9/2015 8:17,60,4,417,Other_Tweet,Russian,0,NonEnglish,В Петербурге появилось вегетарианское кафе в к...


In [19]:
# troll_02[troll_02['retweet']==1]['content without http']

'BREAKING: The audio of sexual predator Harvey Weinstein admitting to assaulting model Ambra Gutierrez has leaked.   I am truly disgusted. '

# WE need to do some formatting of these data for NLP!

+ Depending on our analysis dictates what we do...

In [64]:
eng_twts=troll_02[troll_02['language']=='English']
eng_twts.head()

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http
0,10_GOP,Unknown,English,10/1/2017 19:58,1052,9636,253,Other_Tweet,Right,0,RightTroll,We have a sitting Democrat US Senator on trial...
1,10_GOP,Unknown,English,10/1/2017 22:43,1054,9637,254,Other_Tweet,Right,0,RightTroll,Marshawn Lynch arrives to game in anti-Trump s...
2,10_GOP,Unknown,English,10/1/2017 22:50,1054,9637,255,RETWEET,Right,1,RightTroll,Daughter of fallen Navy Sailor delivers powerf...
3,10_GOP,Unknown,English,10/1/2017 23:52,1062,9642,256,Other_Tweet,Right,0,RightTroll,JUST IN: President Trump dedicates Presidents ...
4,10_GOP,Unknown,English,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,"19,000 RESPECTING our National Anthem! #StandF..."


# `Create Hashtag and Retweet columns:`

+ We have some simple solutions but there are issues:

This works but provides us with only first occurence
`troll_02['content without http'].str.extract(r'@(\S+)')`

If we need a to find more: use `.extractall()` unfortunately it returns multi-index table and difficult to use.


Therefore, I used a loop to deal with this. Unless, you have a good work around beaware of limitations as well as your options.

In [ ]:
# troll_02['content without http'].str.extractall(r'@(\S+)')[0]
# tweets['RT_source'].fillna('None',inplace=True)


In [27]:
troll_02.loc[7,'content without http']

'@SenatorMenendez @CarmenYulinCruz Doesnt matter that CNN doesnt report on your crimes. This wont change the fact that youre going down.'

In [65]:
# Use this to parse Hashtags and @user

def tweet_col_regx(df_col,re_pat):
    q=[]
    for i in df_col:
        if type(i)==str and re.findall(re_pat,i):
            q.append(re.findall(re_pat,i))
        else:
            q.append('None')
    return q


In [66]:
tweet_ref=tweet_col_regx(troll_02['content without http'],r'@(\S+)')

troll_02['tweet_ref']=tweet_ref

In [68]:

hashtag_ref=tweet_col_regx(troll_02['content without http'],r'#(\S+)')

troll_02['hashtag_ref']=hashtag_ref

In [70]:
troll_02.head(8)

,author,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http,tweet_ref,hashtag_ref
0,10_GOP,Unknown,English,10/1/2017 19:58,1052,9636,253,Other_Tweet,Right,0,RightTroll,We have a sitting Democrat US Senator on trial...,[nedryun],None
1,10_GOP,Unknown,English,10/1/2017 22:43,1054,9637,254,Other_Tweet,Right,0,RightTroll,Marshawn Lynch arrives to game in anti-Trump s...,None,None
2,10_GOP,Unknown,English,10/1/2017 22:50,1054,9637,255,RETWEET,Right,1,RightTroll,Daughter of fallen Navy Sailor delivers powerf...,None,[BoycottNFL]
3,10_GOP,Unknown,English,10/1/2017 23:52,1062,9642,256,Other_Tweet,Right,0,RightTroll,JUST IN: President Trump dedicates Presidents ...,None,None
4,10_GOP,Unknown,English,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,"19,000 RESPECTING our National Anthem! #StandF...",None,[StandForOurAnthem🇺🇸]
5,10_GOP,Unknown,English,10/1/2017 2:47,1050,9644,247,Other_Tweet,Right,0,RightTroll,Dan Bongino: Nobody trolls liberals better tha...,None,None
6,10_GOP,Unknown,English,10/1/2017 2:48,1050,9644,248,RETWEET,Right,1,RightTroll,🐝🐝🐝,None,None
7,10_GOP,Unknown,English,10/1/2017 2:52,1050,9644,249,Other_Tweet,Right,0,RightTroll,@SenatorMenendez @CarmenYulinCruz Doesnt matte...,"[SenatorMenendez, CarmenYulinCruz]",None


# `Citations & Help:`

# ◔̯◔

https://www.kaggle.com/auntiebody/coordinating-influence-on-twitter (very good notebook)

https://www.kaggle.com/laurenliz22/russian-troll-tweets (other motivation)